<h1>Intro</h1>
On June 17, 1983 The Police released "Synchronicity," an album which twice interrupted Michael Jackson's "Thriller" at the top of the Billboard charts. Sychronicity was my favorite album when it was released, and remains one of my favorites today.
<p>
This project looks at the staying power of this 40-year-old record. Minimally, I'll look at record sales. But I hope to uncover insight of deeper engagement which might predict longer-term durability: how often are lyrics discussed, how many musicians trying to learn these songs?

<h1>Questions</h1>
<ol>
<li>Which tracks have shown users discussing lyrics? How has this changed over time?
<li>Which songs have been covered? How has this changed over time?
    <li>How have record sales/followers for Synchronicity compared with more recent albums by former members of The Police?
</ol>

<h1>Covers- Web Scraping Second Hand Songs</h1>
This could've been done by API, but I'm web scraping to build my skills

To run this with other albums:
<ul>
    <li>swap URLs in the first step
    <li>update data cleaning to correct the first year
    <li>export with a unique name
</ul>

In [1]:
#Synchronicity(The Police)
url_album = "https://secondhandsongs.com/release/413"
release_year = 1983
bandname = "police" #for file export

#Business as Usual (Men at Work)
#url_album = "https://secondhandsongs.com/release/28748"
#release_year = "1981"
#bandname = "men_at_work" #for file export

#Thriller (Michael Jackson)
# url_album = "https://secondhandsongs.com/release/273"
# release_year = "1982"
# bandname = "jackson" #for file export

#Can’t Slow Down (Lionel Ritchie)
# url_album = "https://secondhandsongs.com/release/2306"
# release_year = "1983"
# bandname = "lionel" #for file export

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
url = url_album
http_response = requests.get(url)
html = http_response.text
soup_songlist = BeautifulSoup(html)

<h3>select the list of songs, then create a list of corresponding URLs</h3>

In [4]:
list_song_urls = []
#loop all html links in the striped table, and grab urls
for html_link in soup_songlist.select(".field-title a"):
    list_song_urls.append("https://secondhandsongs.com"+str(html_link["href"]))

#remove the header row
list_song_urls.pop(0)

#remove any submissions that aren't yet verified
for song_url in list_song_urls:
    if song_url.find("submission") != -1:
        list_song_urls.remove(song_url)

list_song_urls

['https://secondhandsongs.com/performance/656',
 'https://secondhandsongs.com/performance/23164',
 'https://secondhandsongs.com/performance/326045',
 'https://secondhandsongs.com/performance/35963',
 'https://secondhandsongs.com/performance/71166',
 'https://secondhandsongs.com/performance/2204',
 'https://secondhandsongs.com/performance/52439',
 'https://secondhandsongs.com/performance/173743',
 'https://secondhandsongs.com/performance/11041']

<h1>For each song, count the number of covers per year</h1>
would also like to look at song name, written by, language<br>
but for now, to keep it simple

In [5]:
#create an empty data frame
df_covers = pd.DataFrame ({'song':[],'year':[]})
df_covers

,song,year


In [6]:
#put it all together
for song_url in list_song_urls:
    http_response = requests.get(song_url)
    html = http_response.text
    soup_covers = BeautifulSoup(html)
    song_title = soup_covers.select('.entity-title a')[0].text
    df_next_song = pd.DataFrame([{
        "song": song_title,
        "year": next_date.text[-4:] if next_date.text != "Release date " else ""
    } for next_date in soup_covers.select(".field-date")])
    df_covers=pd.concat([df_covers,df_next_song])

<h3>Manually retrieve "Down Under" by Men at Work</h3>
<br>this song was missing from the album, perhaps because it was separately released as a single

In [7]:
#"Down Under" hack. This song was missing from Men at Work, because it was also released as a single
# df_covers = pd.DataFrame ({'song':[],'year':[]})
# df_covers

# song_url = "https://secondhandsongs.com/performance/60901"
# http_response = requests.get(song_url)
# html = http_response.text
# soup_covers = BeautifulSoup(html)
# song_title = soup_covers.select('.entity-title a')[0].text
# df_next_song = pd.DataFrame([{
#     "song": song_title,
#     "year": next_date.text[-4:] if next_date.text != "Release date " else ""
# } for next_date in soup_covers.select(".field-date")])
# df_covers=pd.concat([df_covers,df_next_song])
# bandname = "men_at_work"

<h3>clean up data</h3>
See https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
for methodology in calculating a 'decades' column using numpy

In [8]:
#remove blank years
df_covers=df_covers.drop(df_covers.query("year==''").index)
df_covers["year"] = df_covers["year"].astype(int)

In [9]:
#calculate decade
conditions = [
    (df_covers['year'] <=1992),
     (df_covers['year'] <=2002),
    (df_covers['year'] <=2012),
    (df_covers['year'] <=2022)
]
values = [1,2,3,4]
df_covers['decade'] = np.select(conditions,values)
df_covers

,song,year,decade
1,Every Breath You Take,1983,1
2,Every Breath You Take,1983,1
3,Every Breath You Take,1983,1
4,Every Breath You Take,1983,1
5,Every Breath You Take,1983,1
...,...,...,...
59,Wrapped Around Your Finger,2016,4
60,Wrapped Around Your Finger,2019,4
61,Wrapped Around Your Finger,2020,4
62,Wrapped Around Your Finger,2022,4


<h3>tabulate by year</h3>

In [10]:
df_covers_count = df_covers.groupby("song").value_counts().to_frame().reset_index()
df_covers_count

,song,year,decade,count
0,Every Breath You Take,2008,3,18
1,Every Breath You Take,2015,4,17
2,Every Breath You Take,2017,4,16
3,Every Breath You Take,2014,4,15
4,Every Breath You Take,2018,4,14
...,...,...,...,...
161,Wrapped Around Your Finger,2016,4,1
162,Wrapped Around Your Finger,1997,2,1
163,Wrapped Around Your Finger,1996,2,1
164,Wrapped Around Your Finger,1998,2,1


<h3>Pivot by Year</h3>

In [11]:
df_covers_by_year = df_covers_count.pivot_table(index="song",columns="year",values="count")
df_covers_by_year

year,1983,1984,1985,1986,1987,1989,1990,1991,1992,1993,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
song,,,,,,,,,,,,,,,,,,,,,
Every Breath You Take,7.0,3.0,5.0,2.0,NaN,NaN,NaN,3.0,1.0,5.0,...,15.0,17.0,12.0,16.0,14.0,9.0,12.0,6.0,3.0,3.0
King of Pain,1.0,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,1.0,...,1.0,1.0,3.0,2.0,5.0,1.0,1.0,NaN,2.0,NaN
Mother,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Murder by Numbers,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,2.0,NaN
Synchronicity I,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN
Synchronicity II,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,1.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN
Tea in the Sahara,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.0,1.0,1.0,NaN,NaN,NaN,2.0,NaN
Walking in Your Footsteps,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,NaN
Wrapped Around Your Finger,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,1.0,2.0,1.0,1.0,2.0,3.0,2.0,1.0,1.0,NaN


<h3>Pivot by Decade</h3>

In [12]:
df_covers_by_decade = df_covers_count.pivot_table(index="song",columns="decade",values="count",aggfunc="sum")
df_covers_by_decade

decade,0,1,2,3,4
song,,,,,
Every Breath You Take,3.0,21.0,55.0,118.0,114.0
King of Pain,NaN,4.0,15.0,24.0,18.0
Mother,NaN,1.0,1.0,1.0,1.0
Murder by Numbers,NaN,2.0,3.0,1.0,8.0
Synchronicity I,NaN,1.0,7.0,4.0,4.0
Synchronicity II,NaN,1.0,9.0,12.0,4.0
Tea in the Sahara,NaN,2.0,8.0,11.0,7.0
Walking in Your Footsteps,NaN,2.0,2.0,5.0,4.0
Wrapped Around Your Finger,NaN,3.0,10.0,26.0,17.0


<h3>data cleaning</h3>

In [13]:
#subtract 1 from release year because this counts the original release
#df_covers_by_year[release_year] = df_covers_by_year[release_year] -1
# df_covers_by_decade[1] = df_covers_by_decade[1] -1
#df_covers_by_year

<h3>export</h3>

In [14]:
filename_year = f"{bandname}_covers_by_year.csv"
filename_decade = f"{bandname}_covers_by_decade.csv"
df_covers_by_year.to_csv(filename_year)
df_covers_by_decade.to_csv(filename_decade)